<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/03_02_matrix_factorization_ALS_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка


**Цель:**
- построить и сравнить модель матричной факторизации ALS для implicit feedback, подобрать параметры через Optuna, оценить на валидации и тесте.
**Данные:**
- используем train(разбиение на v_train, val), test  из предыдущих ноутбуков.


## 01. Клонируем репозиторий.Подключаем GoogleDrive.

In [ ]:
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project

Cloning into 'Recommender_Systems_project'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 109 (delta 50), reused 50 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 543.78 KiB | 6.11 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
# подключаем диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10797917 sha256=b87b868e14d07c72c4ee86406d654efcdacd11655b98e6c319c79665a5da68ac
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.5 MB/s eta 0:00:00


## 02. Импорты

In [ ]:
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import optuna
from optuna.samplers import TPESampler

import time
import pickle
from sklearn.decomposition import TruncatedSVD

from scipy.sparse import save_npz, load_npz, csr_matrix
from implicit.als import AlternatingLeastSquares

from pprint import pprint
from pathlib import Path
import pandas as pd
import numpy as np
import random

# импортируем нужные функци
from recommender.data_io import train_test_reader # для чтения сохранённых из 001_data_and_eda_1m_proba
from recommender.splitters import df_time_split  # для time_split разбиения df
from recommender.preprocessing import prepare_ui_matrix  # для получения матрицы взаимодействий
from recommender.metrics import hitrate_at_k, coverage_at_k, precision_at_k,\
                     recall_at_k, ndcg_at_k, map_at_k, model_evaluation
from recommender.results_logger import save_experiment_results # для сохранения результатов



RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

DATA =   Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA/"raw"
PROCESSED = DATA/"processed"
RESULTS_DIR = DATA / "results"
DATA.mkdir(exist_ok=True)
TOP_N = 10

/usr/local/lib/python3.12/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/tmp/pip-install-a37f6t12/implicit_c787d5a977bd4a23b0c5a36829da8b25/./implicit/gpu/utils.h:71)'
  warnings.warn(


## 03. Грузим train, test, meta_данные.



In [ ]:
train_tast_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'

train, test, meta = train_test_reader(train_tast_path)
pprint(meta, width=80, compact=False)
print(f'\ntrain shape : {train.shape}')
print(f'test shape  : {test.shape}')
print( '\n', '*'*50, '\ntrain.head')
display(train.head(3))
print('\n', '*'*50, '\ntest.head')
display(test.head(3))

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

train shape : (800142, 6)
test shape  : (94842, 6)

 ************************************************** 
train.head


,user_id,item_id,rating,timestamp,title,genres
0,635,1251,4,975768620,8 1/2 (1963),Drama
1,635,3948,4,975768294,Meet the Parents (2000),Comedy
2,635,1270,4,975768106,Back to the Future (1985),Comedy|Sci-Fi



 ************************************************** 
test.head


,user_id,item_id,rating,timestamp,title,genres
0,635,3789,5,975768788,"Pawnbroker, The (1965)",Drama
1,635,2987,5,979141847,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
2,635,2988,4,975769007,Melvin and Howard (1980),Drama


## 04. Бьем Train на v_train и val для подбора гиперпараметров
Используем опять вариант **B** - разбиение по времени(Time-based split).
 Все события до порога в v_train, события после в val. Это гарантирует, что ни одно событие из "будущего" не попадёт в v_train.

In [ ]:
# Бьем Train на v_train и val
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
user_id = 'user_id'
item_id = 'item_id'
time_column = 'timestamp'

# минимальное тёплое число оценок
min_n_reitings = 5
# min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
quantile = 0.85    # квантиль разбиения
# разбиение
v_train, val = df_time_split(train, time_column, columns_to_save,
                             user_id , item_id,
                             min_n_reitings, n, k , quantile)

users_intersection = set(val['user_id']) & set(v_train['user_id'])
print('*'*70)
print(f"\n{len(users_intersection)} юзера встречаются одновременно и в v_train и в val")

# залистим тестовых пользователей
val_users = sorted(users_intersection)

print('\ntrain ')
display(v_train.head(2))
print('\ntest ')
display(val.head(2))

Порог разбиения по времени 974814617
Размеры после разбиения: train 680122 test 120020

Осталось пользователей в train : 4748
Новый размер train: (680108, 6)
Осталось пользователей в test: 211
Новый размер test: (26076, 6)
**********************************************************************

211 юзера встречаются одновременно и в v_train и в val

train 


,user_id,item_id,rating,timestamp,title,genres
0,1272,720,5,974814383,Wallace & Gromit: The Best of Aardman Animatio...,Animation
1,1272,2993,2,974814617,Thunderball (1965),Action



test 


,user_id,item_id,rating,timestamp,title,genres
0,1272,2987,4,974822814,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
1,1272,2989,4,974814713,For Your Eyes Only (1981),Action


## 05.Подготовка матриц для ALS (implicit)
ALS из пакета implicit ожидает пользователь–айтем матрицу предпочтений, обычно бинарную или взвешенную по схеме confidence. Можем использовать матрицу от KNN_like.

In [ ]:
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
v_implicit_train_matrix = load_npz(input_dir / "v_train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index_v.pkl", "rb") as f:
    user2index_v = pickle.load(f)

with open(input_dir / "item2index_v.pkl", "rb") as f:
    item2index_v = pickle.load(f)

with open(input_dir / "index2user_v.pkl", "rb") as f:
    index2user_v = pickle.load(f)

with open(input_dir / "index2item_v.pkl", "rb") as f:
    index2item_v = pickle.load(f)

v_implicit_train_matrix
assert isinstance(v_implicit_train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"

In [ ]:
# заменяем реальные ID на индексы из матрицы обучения модели
val_mapped = val.assign(
    user_id = val["user_id"].map(user2index_v),
    item_id = val["item_id"].map(item2index_v)
)
assert val_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
val_items = val_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(v_train['item_id'].map(item2index_v).dropna().astype(int).unique())

val_users_index = list(val_items.keys())

## 06. Генерация рекомендаций для ALS

In [ ]:
# Генерация рекомендаций для ALS
def recommend_with_als(als_model, train_matrix_csr, users, topn=TOP_N):
    """
    Рекомендации для списка пользователей.
    train_matrix_csr: CSR матрица (users × items), двоичная или взвешенная.
    users: список индексов пользователей (в матрице).
    """
    # implicit ожидает item × user при обучении, но recommend берёт user_items (строка CSR)
    recs = {}
    for u in users:
        user_items = train_matrix_csr[u]  # CSR row
        # фильтрует уже просмотренные айтемы автоматически через user_items
        # filter_already_liked=True по умолчанию у recommend
        recommended, scores = als_model.recommend(u, user_items, N=topn, filter_already_liked_items=True,)
        # формата (item_id, score); берём только item_id
        recs[u] =  recommended
    return recs

## 07. подбор гиперпараметров Optuna
- подбор параметров ALS

In [ ]:
#  Objective для Optuna: подбор параметров ALS
def make_objective_als(val_items, all_items, metric_fn=ndcg_at_k, k_eval=TOP_N):
    def objective(trial):
        factors = trial.suggest_int("factors", 4, 256, log=True)
        regularization = trial.suggest_float("regularization", 1e-5, 1e-1, log=True)
        iterations = trial.suggest_int("iterations", 5, 100, log=True)
        alpha = trial.suggest_float("alpha", 1e-2, 10.0, log=True)

        # Готовим матрицу для ALS: item × user (CSC/CSR не критично, но обычно CSR/COO)
        # Добавим confidence: c = 1 + alpha * r (для бинарной r=1 -> c=1+alpha)
        # В бинарном случае умножим всю матрицу на alpha и добавим 1
        # Простой способ: обучаем на (alpha * X.T) и в параметрах use_cg=True для скорости
        X_ui = v_implicit_train_matrix  # users × items
        X_iu = (X_ui.T).tocsr()         # items × users

        # Строим модель ALS
        als = AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations=iterations,
            random_state=RANDOM_STATE,
            use_cg=True
        )

        # Веса confidence: можно передать как alpha * X_iu (implicit учитывает их как confidence)
        X_conf = X_ui.copy()
        X_conf.data = X_conf.data * alpha  # масштаб confidence

        # Обучение
        als.fit(X_conf)


        # Рекомендации
        rec_items = recommend_with_als(als, X_ui, list(val_items.keys()), topn=k_eval)

        # Метрика
        score = metric_fn(rec_items, val_items, k=k_eval)
        return score
    return objective

In [ ]:
#  Запуск Optuna
n_trials = 50
start = time.time()
sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

study_als = optuna.create_study(direction="maximize", sampler=sampler)
study_als.optimize(make_objective_als(val_items, all_items, metric_fn=ndcg_at_k, k_eval=TOP_N),
                   n_trials=n_trials)
end = time.time()
study_als_time = end - start

print('#'*70)
print("\nЛучшие параметры ALS:", study_als.best_params)
print("Лучший NDCG@10:", study_als.best_value)
print('\n', '*'*70)
print(f"Время выполнения: {study_als_time:.2f} секунд")

[I 2025-12-10 10:20:04,568] A new study created in memory with name: no-name-6cc18b4a-3f5a-4cd6-961a-7b587f222091
/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/44 [00:00<?, ?it/s]

[I 2025-12-10 10:20:25,691] Trial 0 finished with value: 0.29026444870049184 and parameters: {'factors': 17, 'regularization': 0.06351221010640701, 'iterations': 44, 'alpha': 0.6251373574521749}. Best is trial 0 with value: 0.29026444870049184.


  0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-12-10 10:20:26,792] Trial 1 finished with value: 0.3030809536273846 and parameters: {'factors': 7, 'regularization': 4.207053950287933e-05, 'iterations': 5, 'alpha': 3.9676050770529883}. Best is trial 1 with value: 0.3030809536273846.


...

  0%|          | 0/38 [00:00<?, ?it/s]

[I 2025-12-10 10:30:18,595] Trial 48 finished with value: 0.333477328091343 and parameters: {'factors': 4, 'regularization': 0.01182773478560067, 'iterations': 38, 'alpha': 0.10226622961914189}. Best is trial 39 with value: 0.35004367172154705.


  0%|          | 0/87 [00:00<?, ?it/s]

[I 2025-12-10 10:31:07,709] Trial 49 finished with value: 0.26181205924101086 and parameters: {'factors': 30, 'regularization': 0.008064603466302627, 'iterations': 87, 'alpha': 0.41076155521948365}. Best is trial 39 with value: 0.35004367172154705.


######################################################################

Лучшие параметры ALS: {'factors': 5, 'regularization': 0.017213181410356635, 'iterations': 21, 'alpha': 0.5853508719975767}
Лучший NDCG@10: 0.35004367172154705

 **********************************************************************
Время выполнения: 663.15 секунд


In [ ]:
# optuna визуализация
print("OPTUNA for ALS")
optuna.visualization.plot_optimization_history(study_als).show()
optuna.visualization.plot_param_importances(study_als).show()
optuna.visualization.plot_parallel_coordinate(study_als).show()

OPTUNA for ALS


### Выводы:
**Optuna - молодец**
- помогла выбрать параметры для результата гораздо выше среднего  

**Лучшие параметры**  
- factors = 5
- regularization ~ 0.017
- iterations = 21
- alpha ~ 0.585
- Score ≈ 0.350 по NDCG@10   

Оптимум достигается при очень низкой размерности латентного пространства и умеренном числе итераций.    

---
**Роль параметров**

**factors**
- Оптимум в диапазоне 5.
- При росте числа компонент качество падает

**regularization**  
- слабая регуляризация, но всё же нужна для устойчивости.

**iterations**
- Слишком маленькие значения (1-3) не успевают обучить модель.
- Слишком большие  не дают прироста.
- Оптимум - средний диапазон (21).


**alpha**

- Умеренный вес неявных взаимодействий, баланс между популярностью и персонализацией.


## 08. Получение метрик на test

In [ ]:
# Получение метрик на test
# загрузка
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index.pkl", "rb") as f:
    user2index = pickle.load(f)

with open(input_dir / "item2index.pkl", "rb") as f:
    item2index = pickle.load(f)

with open(input_dir / "index2user.pkl", "rb") as f:
    index2user = pickle.load(f)

with open(input_dir / "index2item.pkl", "rb") as f:
    index2item = pickle.load(f)

assert isinstance(train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"
train_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 800142 stored elements and shape (5392, 3662)>

In [ ]:
# заменяем реальные ID на индексы
test_mapped = test.assign(
    user_id = test["user_id"].map(user2index),
    item_id = test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train['item_id'].map(item2index).dropna().astype(int).unique())

In [ ]:
# Вспомним лучшее:
print('Лучшие парамеры ALS :',study_als.best_params)

Лучшие парамеры ALS : {'factors': 5, 'regularization': 0.017213181410356635, 'iterations': 21, 'alpha': 0.5853508719975767}


In [ ]:
best = study_als.best_params
als = AlternatingLeastSquares(
    factors=best.get('factors', 32),
    regularization=best.get('regularization', 1e-2),
    iterations=best.get('iterations', 30),
    random_state=RANDOM_STATE,
    use_cg=True
)

X_conf_full = train_matrix.copy()
alpha_best = best.get('alpha', 1.0)
X_conf_full.data = X_conf_full.data * alpha_best
als.fit(X_conf_full)

# Рекомендации
als_recs = recommend_with_als(als, train_matrix, list(test_items.keys()), topn=TOP_N)
als_results = model_evaluation(
    als_recs, test_items, all_items, TOP_N,
    f'als_factors={best["factors"]}_iter={best["iterations"]}_alpha={best["alpha"]:.1f}_reg={best["regularization"]:.2f}'
)
als_results

  0%|          | 0/21 [00:00<?, ?it/s]

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
als_factors=5_iter=21_alpha=0.6_reg=0.02,0.840909,0.340191,0.043517,0.358804,0.24451,0.095576


## 09. Сохраняем результаты

In [ ]:
# Сохраняем результаты
results_data, json_file, csv_file = save_experiment_results(
    result=als_results,
    model_name=f'als_factors={best["factors"]}_iter={best["iterations"]}_alpha={best["alpha"]:.1f}_reg={best["regularization"]:.2f}',
    meta=meta,
    results_dir=RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: als_factors=5_iter=21_alpha=0.6_reg=0.02_20251210_110349.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: als_factors=5_iter=21_alpha=0.6_reg=0.02
Метка времени: 20251210_110349
Дата оценки: 2025-12-10T11:03:49
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 84.1%
precision@10: 34.02%
recall@10: 4.35%
ndcg@10: 35.88%
map@10: 24.45%
Coverage@10: 9.56%

Последние эксперименты (5 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
0,Most_Popular,0.515550,0.132895,0.015591,0.133171,0.068656,0.002731,20251022_194247,2025-10-22T19:42:47.576490
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,2025-11-08T08:49:43.731391
3,trancated_svd_n_comp=5_n_iter=42,0.851675,0.348206,0.045528,0.365538,0.247310,0.091207,20251116_190936,2025-11-16T19:09:36.863512
4,als_factors=5_iter=21_alpha=0.6_reg=0.02,0.840909,0.340191,0.043517,0.358804,0.244510,0.095576,20251210_110349,2025-12-10T11:03:49.859474


# **Итого:**

## **наблюдения:**
**als, factors=5,iter=21**
- немного хуже чем SVD, сравнимо с userKNN
- **HitRate@10**: 84%
- **Precision@10**: 34.0% .
- **Recall@10:** 4.35% .

**Однако:**
- **Coverage@10**: 9.6% - выше чем у всех моделей, говорит о более широком охвате каталога.

##**Выводы:**

**Truncated SVD** - лидер по качеству ранжирования, что логично: она использует градуированную информацию рейтингов, тогда как **ALS** работает с implicit-версией.  

**ALS** - лучший компромисс между качеством и разнообразием: она немного уступает SVD по точности, но покрывает на 0.44% больше каталога, предлагая менее популярные, но персонализированные фильмы.  
  
**KNN-модели** демонстрируют высокую конкурентоспособность, особенно **UserKNN**, но проигрывают по **Coverage** и требуют хранения полной матрицы схожести - хуже масштабируемы.   
**Итог:**   
**ALS** - сильный кандидат для production, если важны масштабируемость (обучение на sparse-матрицах) и разнообразие рекомендаций.
Однако, если качество ранжирования - главный приоритет, а данные содержат явные рейтинги, Truncated SVD (или её аналоги ) остаётся предпочтительнее.
Низкая оптимальная размерность (factors=5) говорит о "простоте" датасета MovieLens-1M - он не требует сложных deep-моделей для достижения сильных результатов.